In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Это демонстрационный блокнот для создания своей модели Gensim fasttext на основе русскоязычного корпуса. В коде использованы всевозможные решения из открытых источников.

На предыдущем шаге были обработаны текстовые данные. 

1) Несколько файлов в формате .pdf были сконвертированы в формат .txt с помощью команды в bash:

> sudo apt-get update && sudo apt-get install -y xpdf
> for file in *.pdf; do pdftotext "$file" "$file.txt"; done

2) Объединяем текстовые данные в один файл с помощью команды в bash:

> find . -type f -name "*" -exec cat >> wrk.txt {} \;


Теперь с помощью стандартных процедур предобрабатываем текст.

Подгружаем бибилиотеки и данные:

In [1]:
import numpy as np
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

In [1]:
import nltk.data
from nltk import tokenize

In [1]:
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

In [1]:
import nltk

In [1]:
import multiprocessing
from gensim.models.fasttext import FastText
from time import time
import gensim

In [1]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [1]:
tokenizer = nltk.data.load('tokenizers/punkt/russian.pickle')

fp = open('../input/gensim-fasttext-training-corpus-for-demo-only/wrk.txt', encoding='utf-8-sig', errors='ignore')

data = fp.read()

x = tokenizer.tokenize(data)

df = pd.DataFrame(x, columns=['text'])

Добавляем пробелы между словами и знаками препинания:

In [1]:
text = df["text"].str.replace(r'([^\w\s]+)', ' \\1 ').str.strip()

Смотрим на характеристики текстовых данных:

In [1]:
type(text)

In [1]:
len(text)

In [1]:
text[:20]

Заметно, что данные нужно как следует обработать.

Приводим данные в нужный нам строковый формат:

In [1]:
text = text.astype(str)

Приводим слова к нижнему регистру:

In [1]:
text = text.str.lower()

In [1]:
text = " ".join(map(str, (review for review in text)))
print ("There are {} words in the combination of all review.".format(len(text)))

Теперь токенизируем текст:

In [1]:
WORD = re.compile(r'\w+')
def regTokenize(text):
    words = WORD.findall(text)
    return words

In [1]:
text_tokens = regTokenize(text)

In [1]:
text = nltk.Text(text_tokens)
print(type(text))
text[:10]

Теперь данные выглядят заметно лучше.

In [1]:
text_raw = " ".join(text)

In [1]:
textfile = open('text.txt', 'w')
textfile.write(text_raw)
textfile.close()

Загружаем данные для обучения модели:

In [1]:
data = gensim.models.word2vec.LineSentence('text.txt')

И обучаем модель с нужными нам гиперпараметрами (в примере используются стандартные значения)

In [1]:
%%time
ft_model = FastText(data, vector_size=100, window=5, min_count=5, workers=4,sg=1)

Если нам нужно дообучить модель, то вместо кода в ячейке выше нужно использовать это:

> %%time

> ft_model = FastText.load("ft_model.model")

> t = time()

> ft_model.build_vocab(data, progress_per=10000, update=True)

> print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

> ft_model.train(data, total_examples=ft_model.corpus_count, epochs=5, report_delay=1)

> print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

Сохраняем модель:

In [1]:
ft_model.save("ft_model.model")

Теперь мы можем использовать модель по назначению: проверить слова на сходство, визуализировать группы слов и так далее:

In [1]:
ft_model.wv.most_similar(positive=["факт"], topn=20)

In [1]:
ft_model.wv.similarity("факт", 'теория')

In [1]:
ft_model.wv.doesnt_match(["субъект", 'объект', 'артефакт'])

In [1]:
ft_model.wv.most_similar(positive=["субъект", 'объект'], negative=['артефакт'], topn=20)

In [1]:
def tsne_plot(labels, tokens, classes, clusters):
    tsne_model = TSNE(perplexity=15, n_components=2, init='pca', n_iter=3500, random_state=33)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []

    for value in new_values:
        x.append(value[0])
        y.append(value[1])

    colors = cm.rainbow(np.linspace(0, 1, clusters))
    
    plt.figure(figsize=(16, 9))
    for i in range(len(x)):
        plt.scatter(x[i], y[i], c=colors[classes[i]].reshape(1,-1), alpha=0.75)
        plt.annotate(labels[i], alpha=0.75, xy=(x[i], y[i]), xytext=(5, 2), 
                     textcoords='offset points', ha='right', va='bottom', size=10)
        
    plt.grid(True)
    plt.savefig('embedding.png', dpi=300)
    plt.show()

In [1]:
labels = []
tokens = []
classes = []

words = ['теория',  'факт', 'субъект', 'объект', 'артефакт']
                   
words.sort()

samples = len(words)
for i, word in enumerate(words):
    tokens.append(ft_model.wv[word])
    labels.append(word.upper())
    classes.append(i)
    for similar_word, similarity in ft_model.wv.most_similar(word, topn=10):
        tokens.append(ft_model.wv[similar_word])
        labels.append(similar_word)
        classes.append(i)

In [1]:
tsne_plot(labels, tokens, classes, samples)